In [8]:
import sqlite3
import pandas as pd
import re
import keras
from unicodedata import normalize
import utils.functions as custom_functions

In [9]:
import imp
imp.reload(custom_functions)

<module 'utils.functions' from '/home/migueljaime/generative_language_model/utils/functions.py'>

In [10]:
sqlite_file = 'data/database.sqlite'
db = custom_functions.create_connection(sqlite_file)

In [11]:
# get a small sample
small_sample = custom_functions.get_sample(nrows=50000, db=db)
display(small_sample)

# save as csv
small_sample.to_csv('data/reddit_small_sample.csv', index = False)

,subreddit,ups,downs,score,body,id,name,link_id,parent_id
0,smashbros,1,0,1,don't diss the grim puncher,cqug90r,t1_cqug90r,t3_34eznn,cqu4f8j
1,BlackPeopleTwitter,8,0,8,right after we all died from the rapture if my...,cqug9br,t1_cqug9br,t3_34exfj,34exfj
2,GlobalOffensive,1,0,1,deleted,cqug9z6,t1_cqug9z6,t3_34fsaq,cqufpjw
3,gonewildaudio,2,0,2,velkommen,cqugatn,t1_cqugatn,t3_34f8js,34f8js
4,pics,1,0,1,lol,cqugchv,t1_cqugchv,t3_34gluk,34gluk
5,Vivillon,1,0,1,can try to find a polar one for you i have a...,cqugcma,t1_cqugcma,t3_32u48w,32u48w
6,Bass,2,0,2,yeah some of those are just straight up awesom...,cqugcst,t1_cqugcst,t3_34fkaa,cqug9dw
7,nba,3,0,3,i am not a dumb guy by any stretch but i could...,cqugdji,t1_cqugdji,t3_34g7jr,cqud905
8,exo,2,0,2,well dang,cqugf1f,t1_cqugf1f,t3_34fxig,cqug04v
9,funny,1,0,1,he will be known forever in his life as the ki...,cqugfkm,t1_cqugfkm,t3_34fi1c,34fi1c


In [ ]:
unique_ids = set(small_sample.id.unique())
unique_parent_ids = set(small_sample.parent_id.unique())

len(unique_ids & unique_parent_ids)

In [ ]:
# get a random sample of 10M messages, with replacement and parent posts
df_1 = custom_functions.get_sample(nrows=1000000, db=db)
print("df_1 count", df_1.count())
df_2 = custom_functions.get_sample(nrows=1000000, db=db)
print("df_2 count", df_2.count())
#df_3 = custom_functions.get_sample(nrows=2000000, db=db)
#df_4 = custom_functions.get_sample(nrows=2000000, db=db)
#df_5 = custom_functions.get_sample(nrows=2000000, db=db)

In [ ]:
# dedupe subsequent samples
ex_id = set(df_1.id.unique())
df_2, ex_id = custom_functions.remove_duplicates(df_2, ex_id)
print("df_2 updated count", df_2.count())